In [342]:
import requests
import urllib
import re
import json
import pickle
import yaml
import pymongo
import pprint
from datetime import datetime
from crontab import CronTab

In [344]:
#TODO
#номер дата время ссылка
#mongo   PATH: 'http://127.0.0.1:8001/job/test/7/testReport/api/python?pretty=true'
#возможность прокручивать
# конфиг yml      sql alchemia    автообновление время скрипта
#counter yaml sum requirements parse zip


# возможно перенести ид и рк из конфига в бд полностью???

In [371]:
config = yaml.load(open('config.yaml'))
PATH = 'http://127.0.0.1:8001/job/test/7/testReport/api/python?pretty=true'
print(yaml.dump(config))
config['to_sum'][1]

PATH: http://127.0.0.1:8001/job/
name:
  test: {id: 44, pk: 25}
to_sum:
  1: [Tests, SmokeTests]
  2: [FunctionalTests]



['Tests', 'SmokeTests']

In [239]:
obj = eval(urllib.request.urlopen(PATH).read())


In [373]:
for curname in config['name'].keys():
    curpath = config['PATH'] + curname + '/lastCompletedBuild/api/python?pretty=true'
    cur = eval(urllib.request.urlopen(curpath).read())
    if int(cur['id']) != (config['name'][curname]['id']):
            names = []
            res = {"job":{'name': curname,'id':int(cur['id']), 'date': datetime.utcfromtimestamp(cur['timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S')}}
            path = config['PATH'] + curname + '/' + cur['id'] + '/' + curname+ 'Report/api/python?pretty=true'
            obj = eval(urllib.request.urlopen(path).read())
            for i in obj['suites'][0]['cases']:
                name = re.findall(r'\..+\.', i['className'])[0].split('.')[1]
                if name not in res.keys():
                    names.append(name)
                    res[name] = {'passed':0, 'failed':0, 'skipped':0, 'total':0}
                res[name]['total'] += 1
                if i['skipped'] == False:
                    if i['status'] == 'PASSED':
                        res[name]['passed'] += 1
                    else:
                        res[name]['failed'] += 1
                else:
                    res[name]['skipped'] += 1

            for name in names:
                res[name]['succeed'] = round(res[name]['passed']/(res[name]['passed'] + res[name]['failed'])*100, 2)
            
            with open(curname + '.json', 'w') as file:
                      json.dump(res, file)
            
            mongoSave(res,curname)
            
            config['name'][curname]['id'] = int(cur['id'])
            config['name'][curname]['pk'] += 1
            
            with open('config.yaml', 'w') as f:
                yaml.dump(config, f)
                
            print('Done')
    else:
        print('already up to date')
        
        

already up to date


In [275]:
#Todo attach mongo DB
def mongoSave(result, jobname):
    conn = pymongo.MongoClient()
    db = conn.testresults
    coll = db[jobname]
    coll.insert_one(result)

In [323]:
conn = pymongo.MongoClient()
db = conn.testresults
coll = db['test']
c = coll.find_one({"job.id": "17"})
for name in c:
    print(name)

_id
job
Tests


In [318]:
# res = {}
# for i in obj['suites'][0]['cases']:
#     name = re.findall(r'\..+\.', i['className'])[0].split('.')[1]
#     if name not in res.keys() :
#         res[name] = {'passed':0, 'failed':0, 'skipped':0, 'total':0}
#     res[name]['total'] += 1
#     if i['skipped'] == False:
#         if i['status'] == 'PASSED':
#             res[name]['passed'] += 1
#         else:
#             res[name]['failed'] += 1
#     else:
#         res[name]['skipped'] += 1
        
# for name in res.keys():
#     res[name]['succeed'] = round(res[name]['passed']/(res[name]['passed'] + res[name]['failed'])*100, 2)
    
# with open('results.json', 'w') as file:
#           json.dump(res, file)
        
# try:           
#     with open ('data.pickle', 'rb') as f:
#         data_graf = pickle.load(f)
# except:
#     data_graf = []
        
    
# for name in res:
#     data_graf.append(res[name]['succeed'])
# with open ('data.pickle', 'wb') as data:
#     pickle.dump(data_graf, data)
    

In [258]:
res
print(datetime.utcfromtimestamp(cur['timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S'))

2019-04-03 12:29:35


In [288]:
config['name']
for name in config['name'].keys():
    print(config['name'][name])

17


In [339]:
my_cron = CronTab(user = 'alexandroleshko')
my_cron.remove_all()
my_cron.write()